# Basin-hopping algorithm in scipy
The `basin-hopping algorithm` in `scipy` is a metaheuristic for the solution of general optimization problems.

"It has been motivated by problems from Physical Chemistry, in particular the taks of finding stable molecular configurations or configurations with the lowest energy. These task typically have lots of local minima which makes is hard for standard optimization methods because there is a very strong dependency on the initial conditions." [See here](https://www.quora.com/Mathematical-Optimization-What-is-the-basin-hopping-algorithm)

The point of this exercise is to have a look at the algorithm and see what it can do. We will therefore solve the following optimization problem:

\begin{equation}
\begin{array}{ll}
\underset{x_0,x_1}{\text{minimize}} & \cos(14.5x_0 - 0.3) + (x_1 + 0.2) x_1 + (x_0 + 0.2)x_0 \\
\text{subject to} & -1.1 \leq x_0,x_1 \leq 1.1
\end{array}
\end{equation}

## Initialization and first test
First, let's load the routine in and just optimize the objective function without constraints:

In [9]:
# Load the solver
from scipy import optimize
from scipy.optimize import basinhopping
import numpy as np

# Define the objective and initial guess
func = lambda x: np.cos(14.5 * x[0] - 0.3) + (x[1] + 0.2) * x[1] + (x[0] + 0.2) * x[0]
x0=[1.0, 1.0]
minimizer_kwargs = {"method": "L-BFGS-B"}

# Solve the problem and print result
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=200)
print("global minimum: x[0] = %.4f, x[1] = %.4f, f(x0) = %.4f" % (ret.x[0], ret.x[1], ret.fun))

global minimum: x[0] = -0.1951, x[1] = -0.1000, f(x0) = -1.0109


# Now let's add some constraints
To add some constraints, we have to add a so-called acceptance test. This means a test that each solution has to pass to be accepted. To do this, you define the following class:

In [10]:
class MyBounds(object):
    def __init__(self, xmax=[1.1,1.1], xmin=[-1.1,-1.1] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

Now, you can use this class as an acceptance test in `basinhopping`:

In [11]:
mybounds = MyBounds()
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=10, accept_test=mybounds)
print("global minimum: x[0] = %.4f, x[1] = %.4f, f(x0) = %.4f" % (ret.x[0], ret.x[1], ret.fun))

global minimum: x[0] = -0.1951, x[1] = -0.1000, f(x0) = -1.0109


# However....let's play with the constraints
Currently, the constraints are inactive. So let's make them active by setting the lower bound to $0.5$:

In [12]:
class MyBounds2(object):
    def __init__(self, xmax=[1.1,1.1], xmin=[0.5,0.5] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [13]:
mybounds2 = MyBounds2()
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=10, accept_test=mybounds2)
print("global minimum: x[0] = %.4f, x[1] = %.4f, f(x0) = %.4f" % (ret.x[0], ret.x[1], ret.fun))

global minimum: x[0] = 1.0926, x[1] = -0.1000, f(x0) = 0.4159


Wow! The constraints are violated (and by a long shot). However if we look at the optimizer, it thinks it found a good solution:

In [14]:
ret

                        fun: 0.41591968109174626
 lowest_optimization_result:       fun: 0.41591968109174626
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.8191672e-06,  8.8817842e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 45
      nit: 11
   status: 0
  success: True
        x: array([ 1.09260105, -0.09999996])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 465
                        nit: 10
                          x: array([ 1.09260105, -0.09999996])

This is not an uncommon problem (see [here](https://github.com/scipy/scipy/issues/7842) and [here](https://github.com/scipy/scipy/issues/7799)), and this is in a main Python package! Therefore, always be careful with metaheuristics!

## Addendum
The standard method to define constraints in `scipy.minimize` is via a dictionary. In our case, this would be:

In [15]:
xmax = [1.1,1.1]; xmin = [0.5,0.5]
cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - xmin[0]},
        {'type': 'ineq', 'fun': lambda x:  -x[0] + xmax[0]},
        {'type': 'ineq', 'fun': lambda x:  x[1] - xmin[1]},
        {'type': 'ineq', 'fun': lambda x:  -x[1] + xmax[1]})

So addding this to the original problem, we try again:

In [16]:
minimizer_kwargs = {"constraints": cons}
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=10)
print("global minimum: x[0] = %.4f, x[1] = %.4f, f(x0) = %.4f" % (ret.x[0], ret.x[1], ret.fun))

global minimum: x[0] = 0.6634, x[1] = 0.5000, f(x0) = -0.0717
